In [1]:
import numpy as np
from numpy.random import multivariate_normal, rand, uniform, random
from numpy import tril

In [2]:
import pandas as pd
import numpy as np
import scipy.linalg as la
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from scipy.stats import random_correlation

import sys
sys.path.append("../src")

from preprocessing.preproc import *
from validation.CV import *

In [5]:
def data_sim():
    no_features = 50
    no_samples = 300

    mean = uniform(low=-10.0, high=10.0, size=(no_features,))
    eigs = uniform(low=1.0, high=2.0, size=(no_features,))
    s = np.diag(eigs)
    q, _ = la.qr(uniform(low=1.0,high=2.0,size=(no_features,no_features)))
    variance = q.T @ s @ q

    takes = multivariate_normal(mean, variance, no_samples)

    size = [10, 10, 10]  # remaining p - sum(size) is reference

    X_idx = [np.sum(size[:i]) for i in range(len(size) + 1)]
    X = [takes[:, int(l):int(r)] for l, r in zip(X_idx, X_idx[1:])]

    alpha = [uniform(low=-1.0, high=1.0, size=(s,)) for s in size]
    delta = random(len(size)) * 20

    val = [np.dot(x, a) for x, a in zip(X, alpha)]
    label = [v > d for v, d in zip(val, delta)]

    arr = np.empty(300, int)

    for i in range(300):
        if label[0][i] == True:
            arr[i] = 1
        elif label[1][i] == True:
            arr[i] = 2
        elif label[2][i] == True:
            arr[i] = 3
        else:
            arr[i] = 4

    return takes, arr

In [6]:
X, y = data_sim()

In [8]:
validator(X, y, LogisticRegression(random_state=0, solver='lbfgs',
                         multi_class='multinomial', max_iter=5000), KFold(n_splits=3, shuffle=True))

{'avg_missclass': 0.09000000000000001,
 'avg_acc': 0.91,
 'avg_prec': 0.91,
 'avg_F1': 0.91,
 'avg_scaled_conf_mx': array([[ 94.,  13.],
        [ 14., 179.]]),
 'avg_conf_prob_mx': array([[0.31333333, 0.04333333],
        [0.04666667, 0.59666667]])}